In [1]:
import redis
import queue

class RedisPubSub(object):

    def __init__(self,
                 hostname: str,
                 port: int = 6379):
        self.hostname = hostname
        self.port = port

    def connect(self):
        try:
            self.redis_client = redis.StrictRedis(host=self.hostname, port=self.port, decode_responses=True)
            self.redis_client.ping()
        except ConnectionError:
            print("ConnectionError while trying to connect to Redis@{}:{}".format(self.hostname,
                                                                                  self.port))
            raise
        self.pubsub = self.redis_client.pubsub()    

    def put(self, channel_name, message):
        subscribers = self.redis_client.publish(channel_name, message)
        print(f"Received by {subscribers}")
        return subscribers
    
    def get(self, channel_name, timeout=2):        
        try:
            package = self.pubsub.get_message(ignore_subscribe_messages=True, timeout=timeout)
            if not package:
                raise queue.Empty("Channels empty")            
        except queue.Empty:
            raise
        return package
    
    def subscribe(self, channel_name):
        self.pubsub.subscribe(channel_name)
        
    def unsubscribe(self, channel_name):
        self.pubsub.unsubscribe(channel_name)

In [2]:
# Connect to redis as two separate clients, so that we will have different
# sockets that will buffer independently
pub = RedisPubSub('127.0.0.1')
sub = RedisPubSub('127.0.0.1')
pub.connect()
sub.connect()

In [3]:
pub.put("channel_foo", "message1")

Received by 0


0

In [ ]:
sub.subscribe("channel_foo")
x = pub.put("channel_foo", "message1")

In [5]:
# Call twice to ignore first empty message
sub.get("channel_foo")
sub.get("channel_foo")

{'type': 'message',
 'pattern': None,
 'channel': 'channel_foo',
 'data': 'message1'}

In [6]:
# Here we post two messages which will be marked as received
# because there are registered subscribers at this point
x = pub.put("channel_foo", "message2")
x = pub.put("channel_foo", "message3")

Received by 1
Received by 1


In [7]:
sub.unsubscribe("channel_foo")

In [9]:
# We will receive messages from channel_foo even though we just 
# unsubscribed because these are messages buffered and not yet processed.
sub.get("channel_foo")

{'type': 'message',
 'pattern': None,
 'channel': 'channel_foo',
 'data': 'message3'}